In [1]:
#Import necessary library

import os
import re
import cv2
import numpy as np
from os.path import isfile, join
import matplotlib.pyplot as plt
from utils import *
from darknet import Darknet
import imutils
import numpy as np
import re
from os.path import isfile, join
from skimage.metrics import structural_similarity, mean_squared_error
import easyocr

In [2]:
# Set the location and name of the cfg file
cfg_file = './cfg/yolov3.cfg'

# Set the location and name of the pre-trained weights file
weight_file = './yolov3.weights'

# Set the location and name of the COCO object classes file
namesfile = 'data/coco.names'

# Load the network architecture
m = Darknet(cfg_file)

# Load the pre-trained weights
m.load_weights(weight_file)

# Load the COCO object classes
class_names = load_class_names(namesfile)

# Print the neural network used in YOLOv3
#m.print_network()

In [3]:
#Creating folder to store the results

if not os.path.exists('Result'):
  
  # Create a new directory because it does not exist 
  os.makedirs('Result')

In [4]:
# Converting the video to image frames

import cv2

# Input the name of the video here
video_file_name = 'TEST_2.MP4'
vidcap = cv2.VideoCapture('TEST_2.MP4')
video_name = video_file_name.split('.')[0]
tracking_file_name = 'Result/'+video_name+'_tracking'
success, image = vidcap.read()
frames_count = 0
os.mkdir(video_name)
os.mkdir(tracking_file_name)
while success:
  cv2.imwrite("./"+video_name+"/%d.jpg" % frames_count, image)    
  success, image = vidcap.read()
  #cv2.imshow(image)
  frames_count += 1
    
print('Total Saved Images: ', frames_count)

Total Saved Images:  250


In [5]:
# get file names of the frames
col_frames = os.listdir(video_name)

# sort file names
col_frames.sort(key=lambda f: int(re.sub('\D', '', f)))

# empty list to store the frames
col_images=[]

for i in col_frames:
    # read the frames
    img = cv2.imread('./'+video_name+'/'+i)
    # append the frames to the list
    col_images.append(img)

In [6]:
# Method to identify objects from a image, using YoloV3

def Predict_objects(image, output_image,threshhold):    
    plt.rcParams['figure.figsize'] = [24.0, 14.0]
    
    original_image = image

    # We adjust the image to fit the first layer of the network's input width and height.    
    resized_image = cv2.resize(original_image, (m.width, m.height))

    # Defining the IOU threshold. 
    iou_thresh = threshhold

    # Defining the NMS threshold. 
    nms_thresh = threshhold

    # Identifying objects in the image.
    boxes = detect_objects(m, resized_image, iou_thresh, nms_thresh)

    # Retrieving all the objects found.
    class_name=print_objects(boxes, class_names,a=0)

    #Plotting the image with associated object class labels and bounding boxes. 
    #plot_boxes(output_image, boxes, class_names, plot_labels = True)
    
    height, width, channels = output_image.shape

    objects = []
    i=0
    for box in boxes:
        (centerx, centery, wid, hei) = (box[0]*width, box[1]*height, box[2]*width, box[3]*height)
        objects.append([class_name[i],centerx, centery, wid, hei])
        i=i+1
    
    return output_image,objects

In [7]:
# Method to track a moving image from two consecutive frames of a video

def track_moving_objects(prev,nxt,name):
    grayA = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
    grayB = cv2.cvtColor(nxt, cv2.COLOR_BGR2GRAY)

    diff_image = cv2.absdiff(grayB, grayA)
    # plotting the image after frame differencing
    #plt.show()
    

    # Performing image thresholding
    ret, thresh = cv2.threshold(diff_image, 30, 255, cv2.THRESH_BINARY)

    # plotting image after thresholding
    # plt.imshow(thresh, cmap = 'gray')
    
    # applying image dilation
    kernel = np.ones((3,3),np.uint8)
    dilated = cv2.dilate(thresh,kernel,iterations = 1)

    # plot dilated image
    # plt.imshow(dilated, cmap = 'gray')
    
    valid_cntrs = []
    
    contours, hierarchy = cv2.findContours(thresh.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)

    for i,cntr in enumerate(contours):
        x,y,w,h = cv2.boundingRect(cntr)
        valid_cntrs.append(cntr)
    
    dmy = prev.copy()

    cv2.drawContours(dmy, valid_cntrs, -1, (127,200,0), 2)
#     cv2.line(dmy, (0, 700),(256,80),(100, 255, 255))

    cv2.imwrite("./"+tracking_file_name+"/%s.jpg" % name, dmy)
    #plt.imshow(dmy)

In [8]:
# Method to find the object from original image, that is close to the object that was detected in difference image between
#two frames

def Find_Object(object1,object2):
    for k in object2:
        
        # For this we are comparing the centre points, heightb and width of the objects found in both the frames 
        if (abs(j[1] - k[1]) <= 200 and abs(j[2] - k[2]) <= 200 and abs(j[3] - k[3]) <= 150 and abs(j[4] - k[4]) <= 150):
            return k

In [9]:
# Method to find the License plate number from given vehicle image using ALPR
# For this we will be finding contours(blocks of images) that has 4 sides (rectangle) with text to identify the number plate,
# crop that portion of the image, and then read the text from that cropped image using EasyOCR.

def Find_LicensePlate(image):
    img =image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # plt.imshow(cv2.cvtColor(gray, cv2.COLOR_BGR2RGB))
    
    bfilter = cv2.bilateralFilter(gray, 11, 17, 17) #Noise reduction
    edged = cv2.Canny(bfilter, 30, 200) #Edge detection
    # plt.imshow(cv2.cvtColor(edged, cv2.COLOR_BGR2RGB))
    
    # finding all significant contours (with larger areas)
    
    keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoints)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

    # finding contours(blocks of images) that has 4 sides (rectangle)
    
    location = None
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 10, True)
        if len(approx) == 4:
            location = approx
            break
       
    # plotting the identified contour with bounding box
    
    mask = np.zeros(gray.shape, np.uint8)
    new_image = cv2.drawContours(mask, [location], 0,255, -1)
    new_image = cv2.bitwise_and(img, img, mask=mask)
    # plt.imshow(cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB))

    # Cropping the image to get the number plate part
    (x,y) = np.where(mask==255)
    (x1, y1) = (np.min(x), np.min(y))
    (x2, y2) = (np.max(x), np.max(y))
    cropped_image = gray[x1:x2+1, y1:y2+1]
    # plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))


    # Reading text from license plate using EASY OCR
    reader = easyocr.Reader(['en'])
    result = reader.readtext(cropped_image)
    text = result[0][-2]
    font = cv2.FONT_HERSHEY_SIMPLEX
    res = cv2.putText(img, text=text, org=(approx[0][0][0], approx[1][0][1]+60), fontFace=font, fontScale=1, color=(0,255,0), thickness=2, lineType=cv2.LINE_AA)
    res = cv2.rectangle(img, tuple(approx[0][0]), tuple(approx[2][0]), (0,255,0),3)
    # plt.imshow(cv2.cvtColor(res, cv2.COLOR_BGR2RGB))
    
    return text

In [10]:
# Method to change the predicted license plate text to correct format

def Format_LicenseNumber(number):
    # Number must be a alphanumeric. So remove other characters
    number = re.sub('[^0-9a-zA-Z]+', '', number)
    
    # Maximum characters the number can have is 8
    number = number[:7]
    
    # The alphabet part of the number is seperated from number part using space or '-'. We are using '-'
    n = re.search(r"\d", number)
    number = number[:n.start()] + '-' + number[n.start():]
    
    # All the alphabets must be in upper case
    number = number.upper()
    
    return number

In [11]:
dict_moving_vehicles = {} # Dictionary that has details of all the moving vehicles, including whether it violated or not 
list_details= [] # List that has all the details of the moving vehicle detected in the image
lines = [] # This is list that is used to note down all the observations in all the frame 

# We will be considering 2 consecutive frames at one time, find the difference between them to identify the moving objects alone
# and find objects detected in all the 3 images and determine moving vehicles

next_image=col_images[0]
final_image=col_images[0]

for frame in range(frames_count-1):

    list_moving_vehicles = []
    print("Frame ",frame)
    lines.append("In frame "+str(frame)+":")
    prev_frame=col_images[frame] # The frame that is considered
    next_frame=col_images[frame+1] # The next frame to the one that is considered
    final_image=col_images[frame] # This is the final difference between the 2 frames we get
    
    # Finding the differnce between 2 images to detect moving objects
    
    # Preprocessing the images using Gaussian Blur

    kernel_size = int(prev_frame.shape[1]/50)
    # must be odd if median
    kernel_size += kernel_size%2-1
    img_prev_blurred = cv2.GaussianBlur(prev_frame, (kernel_size, kernel_size), 1.5)
    img_next_blurred = cv2.GaussianBlur(next_frame, (kernel_size, kernel_size), 1.5)

    # Using the Structural Similary Index Metric to find differences between 2 images
    
    s, difference_score = structural_similarity(
        img_prev_blurred, img_next_blurred,
        multichannel=True, full=True, gaussian_weights=True)
    # the diff is the opposite of the similarity
    diff = 1.0-difference_score

    # Renormalising the difference image
    
    diff = cv2.normalize(
        diff, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    diff = diff.astype(np.uint8)
    # plt.imshow(diff)
    
    # Performing object detection over the difference image for the 2 considered frames
    # For these we use very less threshhold value because our goal is to identify objects, and not its class.
    # We will find the object class (vehicle or not) by using the considered 2 frames
    final_image,moving_obj=Predict_objects(diff,diff,0.001)
    
    # When there is no moving object go to next frame
    if moving_obj is None:
        print("No moving object")
        continue
        
    # When the number of  moving objects detected is more than a limited number, then we will not consider that frame
    # because in these cases, a single object is identified as multiple objects and a 
    # single moving object can be considered as mulitple moving objects
    if len(moving_obj)>9:
        continue
    
    # Performing object detection over the difference image for the 2 considered frames with higher threshhold values
    # to get objects with the accurate class labels.
    final_image,prev_det_obj=Predict_objects(prev_frame,final_image,0.4)
    next_image,next_det_obj=Predict_objects(next_frame,next_frame,0.4)
    
    licensePlateNumber = '' # License plate number
    
    
    # print(moving_obj)
    for j in moving_obj:
        
        # finding the object from original image, that is close to the object that was detected in difference image between
        # two frames
        p_obj=Find_Object(j,prev_det_obj)
        n_obj=Find_Object(j,next_det_obj)
        a_obj=p_obj
        
        if p_obj is None:
            if n_obj is None:
                continue
            else:    
                a_obj = n_obj
        
        print(a_obj)
        
        # We consider only moving objects that are vehicles
        if(a_obj[0] in ["truck","car","bus","motorbike"]):
            print("moving vehicle detected")
            lines.append("Moving vehicle detected")
            
            # If the centre point of the vehicle is within the violation zone, we consider thos vehcile to have violated 
            if(a_obj[1]<700 and a_obj[2]<1200):
                print("violation")
                track_moving_objects(prev_frame,next_frame,frame)
                lines.append("Moving vehicle detected with Violation")
                
            # Parameters for cropping out the moving vehicle alone from the image
            left = ((torch.round(a_obj[1]) - (torch.round(a_obj[3])/2))).int() 
            top = (torch.round(a_obj[2]) - (torch.round(a_obj[4])/2)).int()
            right = (torch.round(a_obj[1]) + (torch.round(a_obj[3])/2)).int()
            bottom = (torch.round(a_obj[2]) + (torch.round(a_obj[4])/2)).int()
            
            try:
                # Cropping an image
                cropped_image = prev_frame[torch.IntTensor(top):torch.IntTensor(bottom), torch.IntTensor(left):torch.IntTensor(right)]           
                # plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
                
                # Detecting license plate number using ALPR
                licensePlateNumber = Find_LicensePlate(cropped_image)
            except Exception:
                
                # If ALPR fails to detect the number, we will further crop the vehicle image and send the image directly
                # to EASY OCR to read the text from image
                
                # This is done beacuse in many cases, APLR is unable to identify the license plate
                try:
                    left = ((torch.round(a_obj[1]) - (torch.round(a_obj[3])/4))).int() 
                    top = (torch.round(a_obj[2]) - (torch.round(a_obj[4])/4)).int() 
                    right = (torch.round(a_obj[1]) + (torch.round(a_obj[3])/4)).int() 
                    bottom = (torch.round(a_obj[2]) + (torch.round(a_obj[4])/4)).int()

                    cropped_image = prev_frame[torch.IntTensor(top):torch.IntTensor(bottom), torch.IntTensor(left):torch.IntTensor(right)]
                    # plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
                    reader = easyocr.Reader(['en'])
                    result = reader.readtext(cropped_image)
                    licensePlateNumber = result[0][-2]
                except Exception:
                    pass
                
                pass

            print("License Plate Number: ",licensePlateNumber)
            if len(licensePlateNumber) >= 5:
                try:
                    licensePlateNumber = Format_LicenseNumber(licensePlateNumber)
                except Exception:
                    licensePlateNumber = ''
                    pass    
            else:
                lines.append("License Plate Number not identified")
                licensePlateNumber = ''
            lines.append(licensePlateNumber)
            
            # Storing the details of the moving vehicles in a dictionary
            # example : [1:['car', tensor(941.7037), tensor(530.7382), tensor(1620.4276), tensor(921.5646),8,'CDER-342'],
            #           [v2:['car', tensor(941.7037), tensor(530.7382), tensor(1620.4276), tensor(921.5646),25,'ABC-1234']]
            # It will have all information about the vehicle including the class label, centre points, width, height
            # frame number and predicted license plate number
            # Also it will have the information of whether a vehicle violated or not, using the index name used.
            # v is added in front of the index number for violated vehicles(eg: v1, v2, v8)
            
            a_obj.append(frame)
            a_obj.append(licensePlateNumber)
            
            # If a vehicle is moving in violation zone we will add 
            if len(dict_moving_vehicles) == 0:
                list_details.append(a_obj)
                if(a_obj[1]<700 and a_obj[2]<1200):
                    dict_moving_vehicles["v"+str(len(dict_moving_vehicles)+1)]=list_details
                else:
                    dict_moving_vehicles[str(len(dict_moving_vehicles)+1)]=list_details
            else:
                flag = 0
                for x, y in dict_moving_vehicles.items():
                    if abs(y[len(y)-1][1]-a_obj[1]) <= 500 and abs(y[len(y)-1][2]-a_obj[2]) <= 500 and abs(y[len(y)-1][5]-frame)<=10:
                        y.append(a_obj)

                        dict_moving_vehicles[x]=y
                        if(a_obj[1]<700 and a_obj[2]<1200 and x[0] != 'v'):
                            dict_moving_vehicles["v"+x] = dict_moving_vehicles.pop(x)
                        flag = 1
                        break
                if flag == 0:
                    list_details.append(a_obj)
                    if(a_obj[1]<700 and a_obj[2]<1200):
                        dict_moving_vehicles["v"+str(len(dict_moving_vehicles)+1)]=list_details
                    else:
                        dict_moving_vehicles[str(len(dict_moving_vehicles)+1)]=list_details
            list_details = []

        else:
            print("moving object detected but not vehicle")
            lines.append("moving object detected but not vehicle")

Frame  0


<ipython-input-11-e844724bfe47>:32: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  s, difference_score = structural_similarity(


Frame  1
Frame  2
Frame  3
Frame  4
Frame  5
Frame  6
Frame  7
Frame  8
Frame  9
Frame  10
Frame  11
Frame  12
Frame  13
Frame  14
Frame  15
Frame  16
Frame  17
Frame  18
Frame  19
Frame  20
Frame  21
Frame  22
Frame  23
Frame  24
Frame  25
Frame  26
Frame  27
Frame  28
Frame  29


CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


['car', tensor(179.8255), tensor(628.8857), tensor(330.1605), tensor(828.8596)]
moving vehicle detected
violation
License Plate Number:  
Frame  30
Frame  31


...

['car', tensor(1849.3361), tensor(428.4414), tensor(121.6224), tensor(333.9543)]
moving vehicle detected


CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


License Plate Number:  
Frame  175


CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


['car', tensor(1864.8712), tensor(432.2876), tensor(112.1672), tensor(321.8727)]
moving vehicle detected
License Plate Number:  
Frame  176
Frame  177
['person', tensor(1894.3302), tensor(450.5607), tensor(46.3058), tensor(253.0949)]
moving object detected but not vehicle
Frame  178
Frame  179
Frame  180
Frame  181
Frame  182
Frame  183
Frame  184
Frame  185
Frame  186
Frame  187
Frame  188
Frame  189
Frame  190
Frame  191
Frame  192
Frame  193
Frame  194
Frame  195
Frame  196
Frame  197
Frame  198
Frame  199
Frame  200
Frame  201
Frame  202
Frame  203
Frame  204
Frame  205
Frame  206
Frame  207
Frame  208
Frame  209
Frame  210
Frame  211
Frame  212
Frame  213
Frame  214
Frame  215
Frame  216
Frame  217
Frame  218
Frame  219
Frame  220
Frame  221
Frame  222
Frame  223
Frame  224
Frame  225
Frame  226
Frame  227
Frame  228
Frame  229
Frame  230
Frame  231
Frame  232
Frame  233
Frame  234
Frame  235
Frame  236
Frame  237
Frame  238
Frame  239
Frame  240
Frame  241
Frame  242
Frame  243
F

In [12]:
# Printing the number of vehicles that violated along with their predicted license plate numbers

count=0
violated_number = []
for x,y in dict_moving_vehicles.items():
    number = []
    if x[0] == 'v':
        count=count+1
        for z in y:
            number.append(z[6])
            number = list(filter(None, number))
        if len(number):
            violated_number.append(max(set(number), key = number.count))
print("Number of vehicles violated in video:",count)
print("Predicted Numbers:",violated_number)

# Storing the results of the video in a list
result =[]
result.append("Number of vehicles violated in video: "+ str(count))
result.append("Predicted Numbers:-")
for i in violated_number:
    result.append(i)

Number of vehicles violated in video: 2
Predicted Numbers: ['BSHR-863', 'CDER-342']


In [13]:
# Prints everything observed in each frame

lines

['In frame 0:',
 'In frame 1:',
 'In frame 2:',
 'In frame 3:',
 'In frame 4:',
 'In frame 5:',
 'In frame 6:',
 'In frame 7:',
 'In frame 8:',
 'In frame 9:',
 'In frame 10:',
 'In frame 11:',
 'In frame 12:',
 'In frame 13:',
 'In frame 14:',
 'In frame 15:',
 'In frame 16:',
 'In frame 17:',
 'In frame 18:',
 'In frame 19:',
 'In frame 20:',
 'In frame 21:',
 'In frame 22:',
 'In frame 23:',
 'In frame 24:',
 'In frame 25:',
 'In frame 26:',
 'In frame 27:',
 'In frame 28:',
 'In frame 29:',
 'Moving vehicle detected',
 'Moving vehicle detected with Violation',
 'License Plate Number not identified',
 '',
 'In frame 30:',
 'In frame 31:',
 'Moving vehicle detected',
 'Moving vehicle detected with Violation',
 'License Plate Number not identified',
 '',
 'In frame 32:',
 'Moving vehicle detected',
 'Moving vehicle detected with Violation',
 'License Plate Number not identified',
 '',
 'In frame 33:',
 'Moving vehicle detected',
 'Moving vehicle detected with Violation',
 'License Pla

In [14]:
# Converting the result list to a text file and storing it in folder

textfile = open("./"+tracking_file_name+"/result_file.txt", "w")
for element in result:
    textfile.write(element + "\n")
textfile.close()